In [1]:
import pandas as pd 

In [37]:
import bs4 as bs
import pandas as pd
import requests

r = requests.get(url)

soup = bs.BeautifulSoup(r.text, 'lxml')
parsed_table = soup.find_all('table')[0] 

rows = parsed_table.find_all('tr')

table_data = []
for row in rows:
    row_data = []
    cells = row.find_all('td')
    for c in cells:
        row_data.append(c.text)
        if c.find('a'):
            
            row_data.append(c.find('a')["href"])
    table_data.append(row_data)
    
        
    

df = pd.DataFrame(table_data)
df.columns = ["common_name", "link", "scientific_name", "status_in_uk", "pairs_text", "field_code_1", "field_code_2"]
df.head()

,common_name,link,scientific_name,status_in_uk,pairs_text,field_code_1,field_code_2
0,Capercaillie,http://app.bto.org/birdfacts/results/bob3350.htm,Tetrao urogallus,Re-introduced Breeder,"1,000 - 10,000 Adults",CP,CAPER
1,Black Grouse,http://app.bto.org/birdfacts/results/bob3320.htm,Lyrurus tetrix,Resident Breeder,"1,000 - 10,000 Males",BK,BLAGR
2,Ptarmigan,http://app.bto.org/birdfacts/results/bob3300.htm,Lagopus muta,Resident Breeder,"1,000 - 10,000 Pairs",PM,PTARM
3,Red Grouse,http://app.bto.org/birdfacts/results/bob3292.htm,Lagopus lagopus,Resident Breeder,"100,000 - 500,000 Pairs",RG,REDGR
4,Red-legged Partridge,http://app.bto.org/birdfacts/results/bob3580.htm,Alectoris rufa,Introduced Breeder,"10,000 - 100,000 Territories",RL,RELPA


'http://app.bto.org/birdfacts/results/bob18920.htm'

In [39]:
from scipy.stats.mstats import gmean

import re
pd.options.display.max_rows = 1000
pd.options.display.float_format = '{:,.0f}'.format
expr = re.compile("[+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][+]?\d+)?")
df["pairs_text"] = df["pairs_text"].str.replace(" million", "E06")
df["pairs_text"] = df["pairs_text"].str.replace(" Million", "E06")
df["pairs_text"] = df["pairs_text"].str.replace(",", "")

df["num_arr"] = df["pairs_text"].str.findall(expr)

def arr_to_num(arr):
    
    if type(arr) != list:
        return 0
    
    if len(arr) == 1:
        return float(arr[0])
    
    if len(arr) == 2:
        return gmean([float(arr[0]), float(arr[1])])
        

df["roughly_how_common"] = df["num_arr"].map(arr_to_num)
df = df.drop("num_arr", axis=1)
df.head()

,common_name,link,scientific_name,status_in_uk,pairs_text,field_code_1,field_code_2,rough_number,roughly_how_common
0,Capercaillie,http://app.bto.org/birdfacts/results/bob3350.htm,Tetrao urogallus,Re-introduced Breeder,1000 - 10000 Adults,CP,CAPER,"3,162","3,162"
1,Black Grouse,http://app.bto.org/birdfacts/results/bob3320.htm,Lyrurus tetrix,Resident Breeder,1000 - 10000 Males,BK,BLAGR,"3,162","3,162"
2,Ptarmigan,http://app.bto.org/birdfacts/results/bob3300.htm,Lagopus muta,Resident Breeder,1000 - 10000 Pairs,PM,PTARM,"3,162","3,162"
3,Red Grouse,http://app.bto.org/birdfacts/results/bob3292.htm,Lagopus lagopus,Resident Breeder,100000 - 500000 Pairs,RG,REDGR,"223,607","223,607"
4,Red-legged Partridge,http://app.bto.org/birdfacts/results/bob3580.htm,Alectoris rufa,Introduced Breeder,10000 - 100000 Territories,RL,RELPA,"31,623","31,623"


In [41]:
df.to_csv("the_british_list.csv", index=False)